In [ ]:
# Copyright 2019 Google LLC
# Copyright 2025 Anonymized Authors

# Licensed under the Apache License, Version 2.0 (the "License"); 
# you may not use this file except in compliance with the License. 
# You may obtain a copy of the License at
# https://www.apache.org/licenses/LICENSE-2.0
"""
This notebook evaluates greedy selection in combination with different mutation techniques.

Requirements: 

-This notebook requires that tensorflow and numpy be installed within the 
Python environment you are running this script in. 

"""
import matplotlib.pyplot as plt
import sys; sys.path.append('..')
from utils.nas_utils import load_nasbench, mutate_spec, run_xevolution, greedy_selection, crossover, NASBench, guided_mutation
from utils.plotting import plot_all
from IPython.display import clear_output

nasb = load_nasbench()


In [ ]:
# here, we test out different mutation methods
# our mutation functions have to follow 
# def mutation(nasbench, population) -> mutated_population 

def regular_mutation(nasbench: NASBench, population, mutation_rate=0.5):
    """Mutates each individual of a population."""
    mutated = []
    for p in population:
        new_spec = mutate_spec(p, nasbench, mutation_rate)
        mutated.append(new_spec)
    return mutated


In [ ]:
# comparing different methods

m1 = lambda nasb, crossed_parents: regular_mutation(nasb, crossed_parents, 0.5)
m2 = lambda nasb, crossed_parents: guided_mutation(nasb, crossed_parents)


exp1 = {
    "data": 
    {
        "regular mutation" : [[],"Light Blue"],
        "guided mutation" : [[],"Dark Blue"],
    },
    "config": 
    {
        "budget" : int(1e6),
        "limits" : (0.9365, 0.9435),
        "n" : 100,
        "print_every" : 2,
        "confidence_intervall" : True,
        "pvalue" : 0.05,
        "significant_areas": False,
        "dataset" : "test",
   }
}

budget = exp1["config"]["budget"]
for run in range(exp1["config"]["n"]):
    nasb.reset_budget_counters()
    times, best_valid, best_test = run_xevolution(nasb, greedy_selection, crossover, m1, budget, 50)
    exp1["data"]["regular mutation"][0].append((times, best_valid, best_test))

    nasb.reset_budget_counters()
    times, best_valid, best_test = run_xevolution(nasb, greedy_selection, crossover, m2, budget, 50)
    exp1["data"]["guided mutation"][0].append((times, best_valid, best_test))

    if (run % exp1["config"]["print_every"] == 0):
        clear_output(wait=True)
        fig, ax = plt.subplots()
        plot_all(exp1, ax)
        plt.show()
        print('Running repeat %d' % (run + 1))

clear_output(wait=True)
fig, ax = plt.subplots()
plot_all(exp1, ax)
plt.show()